In [3]:
from process_bigraph import Composite 

from biosimulators_processes import CORE 


CORE.process_registry.registry

Cannot register SimulariumSmoldynStep. Error:
**
No module named 'simulariumio'
**
Cannot register MongoDatabaseEmitter. Error:
**
No module named 'simulariumio'
**


{'console-emitter': process_bigraph.composite.ConsoleEmitter,
 'ram-emitter': process_bigraph.composite.RAMEmitter,
 'composite': process_bigraph.composite.Composite,
 'dfba-process': biosimulators_processes.processes.dfba_process.DynamicFBA,
 'copasi-process': biosimulators_processes.processes.copasi_process.CopasiProcess,
 'tellurium-process': biosimulators_processes.processes.tellurium_process.TelluriumProcess,
 'utc-amici': biosimulators_processes.processes.amici_process.UtcAmici,
 'utc-copasi': biosimulators_processes.processes.copasi_process.UtcCopasi,
 'utc-tellurium': biosimulators_processes.processes.tellurium_process.UtcTellurium,
 'smoldyn-process': biosimulators_processes.processes.smoldyn_process.SmoldynProcess,
 'smoldyn-io-process': biosimulators_processes.processes.smoldyn_process.SmoldynIOProcess,
 'copasi-step': biosimulators_processes.steps.ode_simulation.CopasiStep,
 'tellurium-step': biosimulators_processes.steps.ode_simulation.TelluriumStep,
 'amici-step': biosimu

In [ ]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

doc = {
    'dFBA': {
        '_type': 'process',
        'address': 'local:dfba-process',
        'config': {
            'model': {
                'model_source': model_fp
            },
            'simulator': 'copasi',
            'start': 0,
            'stop': 10,
            'steps': 100
        },
        'inputs': {},
        'outputs': {
            'solution': ['solution_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'solution': 'tree[float]'
            }
        },
        'inputs': {
            'solution': ['solution_store']
        }
    }
}


spec = doc.copy()